# Example usage of fneighcf

This Ipython notebook illustrates the usage of the [fneighcf](https://github.com/david-cortes/fneighcf) package, a Python implementation of the collaborative filtering algorithm described in _Koren, Y. (2010). Factor in the neighbors: Scalable and accurate collaborative filtering. ACM Transactions on Knowledge Discovery from Data (TKDD), 4(1), 1._.

Loading the [Movielens-100k dataset](https://grouplens.org/datasets/movielens/100k/):

In [1]:
import pandas as pd, numpy as np, time
from datetime import datetime

ratings=pd.read_table('u.data',sep='\t',engine='python',names=['UserId','ItemId','Rating','Timestamp'])
ratings['Timestamp']=ratings.Timestamp.map(lambda x: datetime(*time.localtime(x)[:6])).map(lambda x: pd.to_datetime(x))
ratings=ratings.sort_values(['UserId','ItemId']).reset_index(drop=True)
ratings.head()

,UserId,ItemId,Rating,Timestamp
0,1,1,5,1997-09-23 01:02:38
1,1,2,3,1997-10-15 08:26:11
2,1,3,4,1997-11-03 09:42:40
3,1,4,3,1997-10-15 08:25:19
4,1,5,3,1998-03-13 03:15:12


Splitting the data into a training and testing set - the model will be evaluated only on users and items that were used to estimate the parameters:

In [2]:
time_cutoff='1998-01-01'
train=ratings.loc[ratings.Timestamp<=time_cutoff]
test=ratings.loc[ratings.Timestamp>time_cutoff]
users_train=set(list(train.UserId))
items_train=set(list(train.ItemId))
test=test.loc[test.UserId.map(lambda x: x in users_train)]
test=test.loc[test.ItemId.map(lambda x: x in items_train)]
print(train.shape)
print(test.shape)

(52884, 4)
(5835, 4)


Example usage:

In [3]:
from fneighcf import FNeigh

recc=FNeigh(use_biases=True,norm_nratings=-0.5, reg_param_biases=0.001, reg_param_interactions=0.5, save_ratings=True)
recc.fit(train, maxiter=15, step_size_biases=0.001, step_size_interactions=0.05,
            decrease_step_sqrt=True, use_sgd=False, verbose=True)

Iteration 1
RMSE after biases only: 0.909330883651
RMSE before update:  0.909330883651

Iteration 2
RMSE after biases only: 0.905166672344
RMSE before update:  0.81647693693

Iteration 3
RMSE after biases only: 0.903919292093
RMSE before update:  0.767220757476

Iteration 4
RMSE after biases only: 0.903393038378
RMSE before update:  0.741463320362

Iteration 5
RMSE after biases only: 0.903185178499
RMSE before update:  0.729500018366

Iteration 6
RMSE after biases only: 0.903098627183
RMSE before update:  0.72442912285

Iteration 7
RMSE after biases only: 0.903064449069
RMSE before update:  0.722389008941



Can also fit the model using stochastic gradient descent (iterating over the ratings in a random order, updating the parameters immediately after calcualting errors for each rating) - this requires fewer passes over the data, thus might take less time (recommended for larger datasets). Note that the step sizes and biases should be smaller than when using full gradient descent as above:

In [4]:
recc=FNeigh(use_biases=True,norm_nratings=-0.5, reg_param_biases=0.001, reg_param_interactions=0.001, save_ratings=True)
recc.fit(train, maxiter=5, step_size_biases=0.001, step_size_interactions=0.01,
            decrease_step_sqrt=True, use_sgd=True)

Test set RMSE

In [5]:
test['Predicted']=test.apply(lambda x: recc.predict(x['UserId'],x['ItemId']),axis=1)
np.sqrt(np.mean((test.Rating-test.Predicted)**2))

1.0617309818726695

Can also ignore the item bias (a measure of the overall item popularity) when making recommendations - these are overall lower quality, but more varied and more 'customized' for each user, having more serendipity (i.e. more likely to recommend something not obvious):

In [6]:
test['PredictedNoBias']=test.apply(lambda x: recc.score(
        rating_history=train[['ItemId','Rating']].loc[train.UserId==x['UserId']],
        item=x['ItemId'],
        user=x['UserId'],
        use_item_bias=False), axis=1)

C:\ipython\fneigh\fneigh.py:375: UserWarning: Fewer than two of the items rated had parameters in the model.
  warnings.warn('Fewer than two of the items rated had parameters in the model.')


Recommendations for a user without rating - i.e. based on global item popularity:

In [7]:
%%time
print(recc.top_n(rating_history=[],n=20))

[1499, 1536, 1629, 1467, 1599, 1512, 1104, 1500, 119, 851, 1450, 1405, 1189, 1175, 1144, 1233, 1158, 1449, 272, 1193]
Wall time: 8 ms


Now a fictional user with some random ratings:

In [8]:
%%time
ratings_fictional_user=[(i,np.random.randint(low=1,high=6)) for i in range(1,10) if i in items_train] # (ItemId,Rating)
print(recc.top_n(rating_history=ratings_fictional_user,n=20,user=1))

[1499, 1536, 1629, 1467, 1599, 1512, 1104, 1500, 119, 851, 1450, 1405, 1189, 1175, 1233, 1144, 1158, 64, 1449, 272]
Wall time: 22 ms


Adding more ratings for the fictional user:

In [9]:
%%time
ratings_fictional_user+=[(i,np.random.randint(low=1,high=6)) for i in range(1,100) if i in items_train]
print(recc.top_n(rating_history=ratings_fictional_user,n=20,user=1))

[1499, 1536, 1629, 1467, 1599, 1512, 1500, 1104, 119, 851, 1450, 1405, 1189, 1175, 1233, 1144, 1158, 272, 1449, 169]
Wall time: 43 ms


Recommendations from some random users - a bad point about this model is that Top-N recommendations tend to be all too similar for different users:

In [10]:
%%time
print(recc.top_n_saved(user=1,n=20))

[1536, 1499, 1629, 1467, 1599, 1512, 1104, 1500, 851, 1450, 1405, 1189, 1175, 1144, 1233, 1158, 1449, 272, 1193, 733]
Wall time: 25 ms


In [11]:
print(recc.top_n_saved(user=870,n=20))

[1536, 1499, 1629, 1467, 1599, 1512, 1104, 1500, 119, 851, 1450, 1405, 1189, 1175, 1144, 1233, 1158, 1449, 272, 1193]


If we exclude the item popularity from the scoring of items, the recommendations become more varied and tailored to each user:

In [12]:
%%time
print(recc.top_n_saved(user=1,n=20,use_item_bias=False))

[1682, 694, 652, 654, 655, 656, 657, 663, 664, 692, 693, 695, 706, 696, 697, 698, 699, 700, 701, 702]
Wall time: 30 ms


In [13]:
print(recc.top_n_saved(user=870,n=20,use_item_bias=False))

[1682, 694, 652, 656, 664, 695, 706, 696, 698, 700, 701, 702, 703, 629, 617, 615, 611, 686, 687, 688]


Some basic comparisons:

In [14]:
avg_ratings=train.groupby('ItemId')['Rating'].mean().to_frame().rename(columns={"Rating":"AvgRating"})
test=pd.merge(test,avg_ratings,left_on='ItemId',right_index=True,how='left')

print('Averge movie rating:',test.groupby('UserId')['Rating'].mean().mean())
print('Average rating for top-5 rated by each user:',test.sort_values(['UserId','Rating'],ascending=False).groupby('UserId')['Rating'].head(5).mean())
print('Average rating for bottom-5 rated by each user:',test.sort_values(['UserId','Rating'],ascending=True).groupby('UserId')['Rating'].head(5).mean())
print('Average rating for top-5 recommendations of best-rated movies:',test.sort_values(['UserId','AvgRating'],ascending=False).groupby('UserId')['Rating'].head(5).mean())
print('----------------------')
print('Average rating for top-5 recommendations from this model:',test.sort_values(['UserId','Predicted'],ascending=False).groupby('UserId')['Rating'].head(5).mean())
print('Average rating for top-5 recommendations from this model (without item bias):',test.sort_values(['UserId','PredictedNoBias'],ascending=False).groupby('UserId')['Rating'].head(5).mean())
print('Average rating for bottom-5 (non-)recommendations from this model:',test.sort_values(['UserId','Predicted'],ascending=True).groupby('UserId')['Rating'].head(5).mean())

Averge movie rating: 3.5602718818211856
Average rating for top-5 rated by each user: 4.5298621745788665
Average rating for bottom-5 rated by each user: 2.246554364471669
Average rating for top-5 recommendations of best-rated movies: 4.029096477794793
----------------------
Average rating for top-5 recommendations from this model: 4.059724349157734
Average rating for top-5 recommendations from this model (without item bias): 3.9448698315467077
Average rating for bottom-5 (non-)recommendations from this model: 3.001531393568147


In [15]:
test.sort_values(['UserId','Predicted'],ascending=False).groupby(['UserId']).head(3)

,UserId,ItemId,Rating,Timestamp,Predicted,PredictedNoBias,AvgRating
99919,943,318,3,1998-02-28 06:11:33,4.258205,0.113592,4.526946
99866,943,98,5,1998-02-28 06:09:40,4.184846,0.147542,4.360000
99850,943,56,5,1998-02-28 06:14:29,4.113238,0.229911,4.160000
97905,921,174,5,1998-01-13 08:43:00,3.965752,0.224779,4.323529
97903,921,172,4,1998-01-13 08:43:43,3.890981,0.197802,4.238806
97955,921,603,3,1998-01-13 08:44:28,3.750537,0.053840,4.410256
97716,919,272,5,1998-01-17 19:50:52,4.273774,0.007985,4.250000
97743,919,313,5,1998-01-17 19:50:00,3.995747,0.025801,4.269231
97739,919,305,4,1998-01-17 19:53:43,3.801727,-0.004598,4.000000
97069,913,258,4,1998-03-08 06:24:09,3.841525,0.066703,3.910204
